In [ ]:
# CRIA O AMBIENTE VIRTUAL

conda create -n FFT_NaoSupervisionado python=3.11
conda activate FFT_NaoSupervisionado


In [ ]:
# INSTALA AS DEPENDENCIAS

pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
pip install transformers==4.41.2 datasets==2.19.1 accelerate==0.30.1 tokenizers==0.19.1

pip install ipykernel
pip install bitsandbytes
python -m ipykernel install --user --name=FFT_NaoSupervisionado --display-name="FFT_NaoSupervisionado"

In [2]:
# IMPORTA AS BIBLIOTECAS

import tqdm as notebook_tqdm

import torch
import os

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    pipeline
)

from datasets import load_dataset

/home/ricardo/.conda/envs/FFT_NaoSupervisionado/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# CONFIGURA OS PARÂMETROS E CARREGA O DATASET

MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
DATASET_FILE = "corpus_juridico.txt"
OUTPUT_DIR = "./modelo_juridico_adaptado_v2"

raw_datasets = load_dataset('text', data_files={'train': DATASET_FILE})
print("Dataset carregado com sucesso:")
print(raw_datasets["train"])

Dataset carregado com sucesso:
Dataset({
    features: ['text'],
    num_rows: 812
})


In [4]:
# CARREGAR MODELO E TOKENIZADOR

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Muitos modelos causais não têm um pad_token. Usar o eos_token como substituto
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
print(f"Modelo e o tokenizador carregados: {MODEL_NAME}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Modelo e o tokenizador carregados: Qwen/Qwen2.5-0.5B-Instruct


In [5]:
# PREPARA OS DADOS PARA TREINAMENTO

def tokenize_function(examples):
    # O tokenizador converte o texto em uma sequência de IDs numéricos.
    return tokenizer(examples["text"], truncation=True, padding=False)

tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,            # Processa múltiplos exemplos de uma vez.
    num_proc=os.cpu_count(), # Usa todos os núcleos de CPU disponíveis para acelerar.
    remove_columns=["text"]  # Remove a coluna de texto original, pois não é mais necessária.
)


block_size = 256 # Tamanho do bloco de tokens para treinamento (256, 512, 1024). GPUs com menos memória, usar valor menor.

# Agrupa os textos tokenizados em blocos de tamanho fixo.
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    # Para modelagem de linguagem, os labels (o que o modelo deve prever) são os próprios inputs.
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=os.cpu_count(),
)

print(f"Treinamento será feito em {len(lm_datasets['train'])} blocos de {block_size} tokens.")


Treinamento será feito em 93 blocos de 256 tokens.


In [6]:
# EXECUTA O TREINAMENTO

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) # Responsável por criar os lotes (batches)


# Argumentos de treinamento
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,

    # Hiperparâmetros do Treinamento
    num_train_epochs=10.0,              # Número de vezes que o modelo verá o dataset completo.
    per_device_train_batch_size=8,      # Número de exemplos por lote. Ajuste conforme a VRAM da sua GPU.
    optim="paged_adamw_8bit",           # Usar o otimizador AdamW de 8-bits para economia massiva de VRAM.
    
    # Otimizador e Taxa de Aprendizado
    learning_rate=5e-5,                 # Taxa de aprendizado inicial (padrão robusto para fine-tuning).
    weight_decay=0.01,                  # Regularização L2 para evitar overfitting.
    lr_scheduler_type='cosine',         # Estratégia de decaimento da taxa de aprendizado.
    warmup_steps=50,                    # Passos de aquecimento para estabilizar o início do treino.

    # Salvamento e Checkpoints
    save_strategy="steps",
    save_steps=200,                     # Salvar um checkpoint a cada 200 passos.
    save_total_limit=3,                 # Manter no máximo 3 checkpoints no disco.

    # Logs e Performance
    logging_strategy="steps",
    logging_steps=20,                   # Exibir a perda (loss) a cada 20 passos.
    fp16=torch.cuda.is_available(),     # Usar precisão mista (16-bit) se houver GPU NVIDIA compatível.
)

# Instancia o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    data_collator=data_collator,
)

# Inicia o fine-tuning!
print("Iniciando o fine-tuning de adaptação de domínio...")
trainer.train()

# Salva o modelo final e o tokenizador
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Treinamento concluído. Modelo final gravado em: {OUTPUT_DIR}")

/home/ricardo/.conda/envs/FFT_NaoSupervisionado/lib/python3.11/site-packages/accelerate/accelerator.py:479: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Iniciando o fine-tuning de adaptação de domínio...


 17%|█▋        | 20/120 [00:15<01:14,  1.34it/s]

{'loss': 2.5781, 'grad_norm': 11.455544471740723, 'learning_rate': 1.8e-05, 'epoch': 1.67}


 33%|███▎      | 40/120 [00:29<00:58,  1.37it/s]

...

{'loss': 0.2517, 'grad_norm': 6.355950832366943, 'learning_rate': 3.272542485937369e-05, 'epoch': 6.67}


 83%|████████▎ | 100/120 [01:13<00:14,  1.36it/s]

{'loss': 0.0729, 'grad_norm': 3.6006433963775635, 'learning_rate': 1.122757546369744e-05, 'epoch': 8.33}


100%|██████████| 120/120 [01:28<00:00,  1.36it/s]



{'loss': 0.0175, 'grad_norm': 3.689345121383667, 'learning_rate': 1.006426501190233e-07, 'epoch': 10.0}
{'train_runtime': 88.2887, 'train_samples_per_second': 10.534, 'train_steps_per_second': 1.359, 'train_loss': 0.8443330151339372, 'epoch': 10.0}
Treinamento concluído. Modelo final gravado em: ./modelo_juridico_adaptado_v2
Treinamento concluído. Modelo final gravado em: ./modelo_juridico_adaptado_v2


In [12]:
# VERIFICAÇÃO E DEMONSTRAÇÃO

# Um prompt para testar a "personalidade" jurídica do modelo
prompt = "Liste 3 aspectos legais da colaboração premiada."

print(f"\n--- [TESTE 1] Modelo Original: {MODEL_NAME} ---")
generator_original = pipeline('text-generation', model=MODEL_NAME, device=0 if torch.cuda.is_available() else -1)
print(generator_original(prompt, max_new_tokens=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)[0]['generated_text'])

print(f"\n--- [TESTE 2] Modelo Adaptado (Pós-Fine-tuning): {OUTPUT_DIR} ---")
generator_adaptado = pipeline('text-generation', model=OUTPUT_DIR, device=0 if torch.cuda.is_available() else -1)
print(generator_adaptado(prompt, max_new_tokens=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)[0]['generated_text'])



--- [TESTE 1] Modelo Original: Qwen/Qwen2.5-0.5B-Instruct ---


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Liste 3 aspectos legais da colaboração premiada. Listar 3 aspectos legais da colaboração premiada, de acordo com a lista fornecida:

1. Legitimidade da premiação: O prêmio é uma forma de reconhecimento e valorização do trabalho realizado por um indivíduo ou equipe.

2. Inclusão de indivíduos com deficiência: As empresas podem conceder prêmios para indivíduos com deficiência, garantindo que todos sejam representados em

--- [TESTE 2] Modelo Adaptado (Pós-Fine-tuning): ./modelo_juridico_adaptado_v2 ---


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Liste 3 aspectos legais da colaboração premiada.A colaboração premiada é uma forma de justiça penal consensual sujeita a limites constitucionais.O instituto reflete o movimento de transformação do processo penal em um sistema cooperativo.A aplicação da colaboração premiada deve respeitar os princípios da legalidade e da moralidade administrativa.A proteção de dados e informações pessoais do colaborador é condição para a integridade do procedimento.A colaboração
